In [ ]:
import os
import sys
import nest_asyncio
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
from trusteval.dimension.privacy.privacy_vlm import pipeline
nest_asyncio.apply()
base_dir = "./test/privacy/"
os.makedirs(base_dir, exist_ok=True)

In [ ]:
from trusteval import download_metadata
download_metadata(
    section='privacy_vlm',
    output_path=base_dir
)

In [ ]:
pipeline.pipeline(base_dir=base_dir)

In [ ]:
from trusteval.src.response_generator.lm_response import generate_responses
import shutil
source_config = "../section/privacy/privacy_vlm/file_config.json"
target_config = os.path.join(base_dir,"output","file_config.json")

if os.path.exists(source_config):
    shutil.copy2(source_config, target_config)
    print(f"Successfully copied file_config to {target_config}")
else:
    print("Warning: Source file_config not found")
    
data_folder  = os.path.join(base_dir,"output")
model_list = [
    'gpt-4o']

await generate_responses(
        data_folder=data_folder,
        request_type="vlm",
        async_list=model_list,
        prompt_key="prompt",
        result_key="responses",
        image_key="image_path"
    )

In [ ]:
from trusteval.src.evaluation import judge_responses

config_path = os.path.join(parent_dir, 'trusteval/src/config/judge_prompt.yaml')
async_judge_model = ['gpt-4o-mini']
response_key = ['responses']
judge_key = 'judge'
judge_type='vlm'

await judge_responses(
        data_folder=data_folder,
        async_judge_model=async_judge_model,
        target_models=model_list,
        judge_type=judge_type
    )

In [ ]:
from trusteval.src.evaluation import lm_metric

lm_metric(
    base_dir=data_folder,
    aspect="privacy_vlm",
    model_list=model_list
)